In [1]:
DIRECTORY='service_ocr'
!mkdir -p $DIRECTORY

## API

In [2]:
%%writefile $DIRECTORY/requirements.txt
opencv-python
google-cloud-documentai
firebase-admin
fastapi
# fastapi==0.92.0
pydantic==1.10.5
uvicorn
python-multipart
# uvicorn==0.20.0
# catboost==1.1.1
numpy==1.21.5
pandas==1.5.2
gunicorn==20.1.0

Overwriting service_ocr/requirements.txt


In [3]:
%%writefile $DIRECTORY/Dockerfile
# Start from a base image
#FROM python:3.9-slim
#FROM python:3.9
#FROM python:3.10-slim
#FROM gcr.io/deeplearning-platform-release/tf-gpu

FROM tensorflow/tensorflow:2.11.0

# Instala las dependencias de OpenCV
RUN apt-get update
RUN apt-get install -y libsm6 libxext6 libxrender-dev
RUN apt-get install -y libgl1-mesa-glx
RUN apt-get install -y libglib2.0-0
RUN apt-get install -y ca-certificates

ENV PYTHONUNBUFFERED True

# Set the working directory
WORKDIR /app

# Copy the requirements file into the container

COPY requirements.txt requirements.txt

# Install the required packages
RUN pip install --upgrade pip

RUN pip install --no-cache-dir -r requirements.txt

# RUN pip install uvicorn

# Copy the application code into the container
# COPY ["loan_catboost_model.cbm", "app.py", "./"] .

# Expose the app port
# EXPOSE 80
ENV PORT=80
EXPOSE $PORT

COPY . ./
# Run command
# CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", $PORT]
CMD exec uvicorn main:app --host 0.0.0.0 --port ${PORT}

Overwriting service_ocr/Dockerfile


In [4]:
%%writefile $DIRECTORY/deploy.sh

#PROJECT_ID="igneous-sandbox-387915"
PROJECT_ID="total-bliss-396122"
REGION='us-central1'
CLOUD_RUN_PROJECT_NAME='tinka-ocr'
# Variables de entorno
VAR1='my-variable'
LOCATION="us"
PROCESSOR_ID="341e773888de340f"
PROCESSOR_VERSION="rc"
MIME_TYPE="image/png"
SECRET_TOKEN="VXZXYMCFUHpEOAE7duiIrFbKCjtTFURof6iGIbXmT1WzK05DhqdC6X2bChvdqGOt"
SERVICE_ACCOUNT='prd-tinka-ocr-ai@total-bliss-396122.iam.gserviceaccount.com'
#SERVICE_ACCOUNT='dev-tinka-ocr-ai-840@igneous-sandbox-387915.iam.gserviceaccount.com'

gcloud run deploy ${CLOUD_RUN_PROJECT_NAME} \
    --region $REGION \
    --project $PROJECT_ID \
    --service-account $SERVICE_ACCOUNT \
    --allow-unauthenticated \
    --cpu=4 \
    --memory=8Gi \
    --min-instances=0 \
    --max-instances=10 \
    --set-env-vars="VAR1=$VAR1" \
    --set-env-vars="SECRET_TOKEN=$SECRET_TOKEN" \
    --set-env-vars="PROJECT_ID=$PROJECT_ID" \
    --set-env-vars="LOCATION=$LOCATION" \
    --set-env-vars="PROCESSOR_ID=$PROCESSOR_ID" \
    --set-env-vars="PROCESSOR_VERSION=$PROCESSOR_VERSION" \
    --set-env-vars="MIME_TYPE=$MIME_TYPE" \
    --source .

# SERVICE_URL=$(gcloud run services describe $CLOUD_RUN_PROJECT_NAME --format 'value(status.url)' --project $PROJECT_ID)
# echo "SERVICE_URL: $SERVICE_URL"

Overwriting service_ocr/deploy.sh


In [17]:
%%writefile $DIRECTORY/utils.py

import base64
import numpy as np
import cv2
from typing import Sequence, Optional, Dict
from google.api_core.client_options import ClientOptions
from google.cloud import documentai
# Revisar lo de SSL
#import ssl
import re
import time

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

from datetime import datetime
import pytz
import uuid

def encontrar_correspondencias_recorte_sift(imagen1, imagen2):
    img1 = cv2.imread(imagen1, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(imagen2, cv2.IMREAD_GRAYSCALE)
    
    print("dimensiones de imagen input: ", img1.shape)
    # Crear detectores y descriptores SIFT
    sift = cv2.SIFT_create()
    #sift = cv2.ORB_create()
    #sift = cv2.xfeatures2d.SURF_create()
    
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Crear un objeto BFMatcher
    bf = cv2.BFMatcher()
    
    # Encontrar correspondencias utilizando BFMatcher
    matches = bf.knnMatch(des1, des2, k=2)

    # Aplicar un umbral para seleccionar las mejores coincidencias
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    
    # Ordenar las coincidencias por distancia
    good_matches = sorted(good_matches, key=lambda x: x.distance)
    
    return good_matches, img1, img2, kp1, kp2, (img2.shape[1], img2.shape[0])

def cropping_and_validation_background(input_img_path, roi1_path, roi2_path):
    good_matches_roi1, img1_roi, _, kp1_roi1, kp2_roi1, shape_roi1 = encontrar_correspondencias_recorte_sift(input_img_path, roi1_path)
    good_matches_roi2, _, _, kp1_roi2, kp2_roi2, shape_roi2 = encontrar_correspondencias_recorte_sift(input_img_path, roi2_path)
    print(f"coincidencias 1: {np.shape(good_matches_roi1)}")
    print(f"coincidencias 2: {np.shape(good_matches_roi2)}")
    
    if((len(good_matches_roi1) <= 4) and (len(good_matches_roi2) <=4)):
        print('> Warning: bajas coincidencias a las dos imagenes de referencia')
        return False
    
    flag = True
    
    if(len(good_matches_roi1)>= len(good_matches_roi2)):
        try:
            #print("antes de homografia 1")
            puntos_img = np.float32([kp1_roi1[m.queryIdx].pt for m in good_matches_roi1]).reshape(-1, 1, 2)
            puntos_referencia = np.float32([kp2_roi1[m.trainIdx].pt for m in good_matches_roi1]).reshape(-1, 1, 2)
            
            unique, counts = np.unique(puntos_referencia[:,:, 1], return_counts=True)
            #print(np.max(counts), counts.shape[0])
            
            if(np.max(counts) > counts.shape[0]):
                print(f"> Warning: Las puntos repetitivos {np.max(counts)} es mayor que las correspondecias {counts.shape[0]} con la imagen de referencia 1")
                flag = False
            
            M, _ = cv2.findHomography(puntos_img, puntos_referencia, cv2.RANSAC, 5.0)
            img1_recortada = cv2.warpPerspective(img1_roi, M, shape_roi1)
        except Exception as e:
            print(e)
            print("> Warning: Las imagenes tienen correlaciones repetitivas , ingresar imagenes encuadras y/o correctas")
            flag = False
    else:
        try:
            #print("antes de homografia 2")
            puntos_img = np.float32([kp1_roi2[m.queryIdx].pt for m in good_matches_roi2]).reshape(-1, 1, 2)
            puntos_referencia = np.float32([kp2_roi2[m.trainIdx].pt for m in good_matches_roi2]).reshape(-1, 1, 2)
            
            unique, counts = np.unique(puntos_referencia[:,:, 1], return_counts=True)
            #print(np.max(counts), counts.shape[0])
            
            if(np.max(counts) > counts.shape[0]):
                print(f"> Warning: Las puntos repetitivos {np.max(counts)} es mayor que las correspondecias {counts.shape[0]} con la imagen de referencia 2")
                flag = False
            
            M, _ = cv2.findHomography(puntos_img, puntos_referencia, cv2.RANSAC, 5.0)
            
            img1_recortada = cv2.warpPerspective(img1_roi, M, shape_roi2)
        except Exception as e:
            print(e)
            print("> Warning: Las imagenes tienen correlaciones repetitivas , ingresar imagenes encuadras y/o correctas")
            flag = False
    return flag

def process_document(
    project_id: str,
    location: str,
    processor_id: str,
    processor_version: str,
    file_path: str,
    mime_type: str,
) -> documentai.Document:
    # You must set the `api_endpoint` if you use a location other than "us".
    client = documentai.DocumentProcessorServiceClient(
        client_options=ClientOptions(
            api_endpoint=f"{location}-documentai.googleapis.com"
        )
    )

    name = client.processor_version_path(
        project_id, location, processor_id, processor_version
    )

    with open(file_path, "rb") as image:
        image_content = image.read()

    request = documentai.ProcessRequest(
        name=name,
        raw_document=documentai.RawDocument(content=image_content, mime_type=mime_type),
    )

    result = client.process_document(request=request)
    return result.document

def ocr_tinka_validation(
    project_id: str,
    location: str,
    processor_id: str,
    processor_version: str,
    file_path: str,
    mime_type: str):
    # Online processing request to Document AI
    document = process_document(
        project_id, location, processor_id, processor_version, file_path, mime_type
    )
    
    text = document.text
    return list(text.split("\n"))

def verificar_contenido(palabra, palabra_contenedora):
    if palabra in palabra_contenedora:
        return True
    else:
        return False

def validar_contenido(valores, list_text):
    print('Textos reconocidos: \n',list_text)
    for text in list_text:
        for valor in valores:
            if (verificar_contenido(valor.lower(), text.lower())==True):
                return True
    return False

def verificar_contiene_numero(texto):
    patron = r'\d'
    coincidencias = re.search(patron, texto)
    if coincidencias:
        return True
    else:
        return False

def fb_fakes_images_validation(input_img_path, roi1_path, list_text, valores):
    good_matches_roi1, _, _, kp1_roi1, kp2_roi1, shape_roi1 = encontrar_correspondencias_recorte_sift(input_img_path, roi1_path)
    
    if(len(good_matches_roi1) <= 4):
        flag_match=False
    else:
        flag_match=True
    
    if(validar_contenido(valores, list_text)==False):
        flag_contenido=True
    else:
        flag_contenido=False
    
    if((flag_match and flag_contenido) or flag_contenido):
        return True
    else:
        return False

def verificar_numero_valido(text):
    count = 0
    for c in text:
        #print(c.isnumeric())
        if (count >3):
            return False
        
        if not c.isnumeric() and c!=' ':
            count+=1
    return True
    
def contenido_validation(pozos_base, list_palabras):
    loterias = []
    for text in list_palabras:
        if(verificar_contiene_numero(text) == True):
            if (verificar_numero_valido(text) == True):
                loterias.append(re.findall(r'\d+\.\d+|\d+', text)[0])
            else:
                loterias.append(text)
    print("pozos basse: \n", pozos_base)
    
    #pozos base
    for p in pozos_base:
        print(f"loteria key: {p} | pozos: {pozos_base[p]}", "tipo: ", type(pozos_base[p][0]))
    
    # Validacion de loterias encontradas, con la BASE
    result_dict = {}
    predict_json = {}
        
    #Asociasion de montos
    for pozo in pozos_base:
        print("----------------------------------------------")
        flag = False
        while len(loterias) > 0:
            try:
                loteria_temp = float(loterias[0])
                print("try", float(loterias[0]))
            except Exception as e:
                #loteria_temp = 0.0                
                print("> Warning: Existen letras en la loteria por comparar, monto descartado")
                print(f'remocion de valor {loterias[0]}')
                print("Exception:" , e, loterias[0])
                del loterias[0]
                continue
                        
            print('posibles loterias: ',loterias)
            print(loterias[0])
            
            print("loteria temp: ", loteria_temp, type(loteria_temp))
            
            print("loteria pozo[0]: ", pozos_base[pozo][0], type(pozos_base[pozo][0]))
            print("loteria pozo[1]: ", pozos_base[pozo][1], type(pozos_base[pozo][1]))
            
            if(float(pozos_base[pozo][0]) == loteria_temp):
                print(f'{pozos_base[pozo][0]} loteria correcta')
                flag = True
                predict_json[pozo] = pozos_base[pozo][0]
                del loterias[0]
                break
            print("no entro a 1er if")
            
            if((pozos_base[pozo][1] == None) or (float(pozos_base[pozo][1]) == loteria_temp)):
                print(f'{pozos_base[pozo][1]} loteria correcta')
                flag = True
                predict_json[pozo] = pozos_base[pozo][1]
                del loterias[0]
                #print('despues: ', loterias)
                break
            
            print("no entro a 2do if")
            print(f'remocion de valor {loterias[0]}')
            del loterias[0]
        #fin de while
        
        result_dict[pozo] = flag
    
    # validacion del resultado
    for k in result_dict:
        if (result_dict[k] == False):
            return False, {}
    
    return True, predict_json

def firestore_document_get(firestore_client, collection, document_id):
    doc_ref = firestore_client.collection(collection).document(document_id)
    doc = doc_ref.get()
    data = doc.to_dict()
    exist = doc.exists
    return data, exist

def firestore_document_insert(firestore_client, collection, data):
    TZ = pytz.timezone('US/Pacific')
    doc_id = str(uuid.uuid4())
    
    data['created'] = datetime.now(TZ)
    data['id_request'] = doc_id
        
    doc_ref = firestore_client.collection(collection).document(doc_id)
    print(doc_ref.set(data))

Overwriting service_ocr/utils.py


In [18]:
%%writefile $DIRECTORY/main.py
from fastapi import FastAPI, UploadFile, File, Header
from fastapi import HTTPException
from pydantic import BaseModel
from utils import *
import os

app = FastAPI(
    title="Tinka Keras-OCR API",
    description="""Validación de los pozos a partir de una imagen""",
    version="0.0.1",
)

# Funcion MAIN (Flujo del Servicio)
def validacion_documentai(project_id, 
                          location, 
                          processor_id, 
                          processor_version, 
                          image_path, 
                          pozos,
                          mime_type, 
                          valores_tinka,
                          valores_actualiza):
    
    img_ref_1 = "images/ROI.png"
    img_ref_2 = "images/ROI2.png"
    img_ref_3 = "images/ROI3.png"
    
    ini_time_cropping = time.time()
    
    val_1 = cropping_and_validation_background(image_path, img_ref_1, img_ref_2)
    
    end_time_cropping = time.time()
    duration_cropping = end_time_cropping - ini_time_cropping
    print("duracion step1: ", duration_cropping)
    
    if (val_1 == False):
        message = "La foto no es válida.  Toma una foto del elemento multipozo. [Tinka:{}/Kabala:{}/Ganagol:{}]".format(pozos['Tinka'][0],pozos['Kabala'][0],pozos['Ganagol'][0])
        print("> "+message)
        return ('False', '1', message, {})
    
    ini_time_ocr = time.time()
    
    list_text = ocr_tinka_validation(project_id=project_id,\
                                     location=location,\
                                     processor_id=processor_id,\
                                     processor_version=processor_version,\
                                     file_path=image_path,\
                                     mime_type=mime_type)
        
    val_2 = validar_contenido(valores_tinka, list_text)
    
    end_time_ocr = time.time()
    duration_ocr = end_time_ocr - ini_time_ocr
    print("duracion step2: ", duration_ocr)
    
    if (val_2 == False):
        message = "La foto no es válida. Enfoca todo el elemento multipozo para validarlo. [Tinka:{}/Kabala:{}/Ganagol:{}]".format(pozos['Tinka'][0],pozos['Kabala'][0],pozos['Ganagol'][0])
        print("> "+message)
        return ('False', '2', message, {})
    
    ini_time_fakes = time.time()
    
    val_3 = fb_fakes_images_validation(image_path, img_ref_3, list_text, valores_actualiza)
    
    end_time_fakes = time.time()
    duration_fakes = end_time_fakes - ini_time_fakes
    print("duracion step3: ", duration_fakes)
    
    if (val_3 == False):
        message = "La foto no es válida. Toma una foto del elemento multipozo de tu Punto de Venta. [Tinka:{}/Kabala:{}/Ganagol:{}]".format(pozos['Tinka'][0],pozos['Kabala'][0],pozos['Ganagol'][0])
        print("> "+message)
        return ('False', '3', message, {})
    
    ini_time_pozos = time.time()
    
    val_4, json_predict = contenido_validation(pozos, list_text)
    
    end_time_pozos = time.time()
    duration_pozos = end_time_pozos - ini_time_pozos
    print("duracion step4: ", duration_pozos)
    
    if (val_4 == False):
        message = "Los pozos son incorrectos. Los pozos correctos son [Tinka:{}/Kabala:{}/Ganagol:{}]".format(pozos['Tinka'][0],pozos['Kabala'][0],pozos['Ganagol'][0])
        print(message)
        return ('False', '4', message, {})
    
    message = "OK los pozos son correctos. Tinka: {}; Kabala:{}; Ganagol:{}".format(json_predict['Tinka'], json_predict['Kabala'], json_predict['Ganagol'])
    print(message)
    return ('True', '1', message, json_predict)

# Home page
@app.get("/")
def home():
    return {"message": "Servicio de reconocimiento de caracteres a partir de imagen, para la Tinka", "model_version": 0.1}

class InputData(BaseModel):
    base64_data: str
    json_data: dict
    id_cliente: str

@app.post("/tinka-ocr/")
async def recognize_pipeline(input_data: InputData, token: str = Header(None)):
    collection = 'tinka-ocr'
    
    if token == SECRET_TOKEN:
        print("Inicio de sesión exitoso, token: ", token)
    else:
        print("Error de autenticación, token: ", token)
        raise HTTPException(status_code=500, detail="Error de autenticación, token inválido")
    # Leyendo data
    im_bytes = base64.b64decode(input_data.base64_data)
    json_data = input_data.json_data
    id_usuario = input_data.id_cliente
    
    if input_data is None:
        data = {
                'id_usuario': id_usuario,
                'validacion': None,
                'tipo': None,
                'duration': None,
                'pozos': json_data,
                'pozos_predict': None,
                'message': 'Error: No se ha proporcionado ningun dato'
                }
        firestore_document_insert(firestore_client, collection, data)
        raise HTTPException(status_code=500, detail="Error: No se ha proporcionado ningun dato")
        
    try:
        im_arr = np.frombuffer(im_bytes, dtype=np.uint8)
        img = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
        
        output_img_path = "images/cropp_img.jpg"
        
        cv2.imwrite(output_img_path, img)
                
        start_time = time.time()
        
        validacion, tipo, mensaje, json_predict = validacion_documentai(project_id=PROJECT_ID,\
                                                            location=LOCATION,\
                                                            processor_id=PROCESSOR_ID,\
                                                            processor_version=PROCESSOR_VERSION,\
                                                            image_path=output_img_path,\
                                                            pozos=json_data,\
                                                            mime_type=MIME_TYPE,\
                                                            valores_tinka=VALUES_TINKA,
                                                            valores_actualiza=VALUES_ACTUALIZA)
        end_time = time.time()
        duration = end_time - start_time
                
        data = {
                'id_usuario': id_usuario,
                'validacion': validacion,
                'tipo': tipo,
                'duration': duration,
                'pozos': json_data,
                'pozos_predict': json_predict,
                'message': mensaje
                }
        
        firestore_document_insert(firestore_client, collection, data)
        
        return {"resultado": validacion, 
                "tipo": tipo, 
                "mensaje": mensaje
               }
    except Exception as e:
        data = {
                'id_usuario': id_usuario,
                'validacion': None,
                'tipo': None,
                'duration': None,
                'pozos': json_data,
                'pozos_predict': None,
                'message': e
                }
        #firestore_document_insert(firestore_client, collection, data)
        print("excepcion encontrada")
        print(e)
        raise HTTPException(status_code=500, detail="Error al procesar los datos, revisar los Logs")

async def startup_event():
    global firestore_client
    global SECRET_TOKEN
    global PROJECT_ID
    global LOCATION
    global PROCESSOR_ID
    global PROCESSOR_VERSION
    global VALUES_TINKA
    global VALUES_ACTUALIZA
    global MIME_TYPE
    print("La aplicación se está iniciando")
    
    VALUES_TINKA = ["tinka", "otinka", "t1nka", "t1nk4", "7inka", "71nka", "71nk4"]
    VALUES_ACTUALIZA = ["actualiza", "4ctualiza", "4ctu4liza", "actual1za", "4ctual1za", "4ctual1z4"]
    
    cred = credentials.ApplicationDefault()
    firebase_admin.initialize_app(cred)
    firestore_client = firestore.client()
    print("firestore cliente: ", firestore_client)
    PROJECT_ID = os.getenv('PROJECT_ID', "total-bliss-396122")
    LOCATION = os.getenv('LOCATION', "us")
    
    PROCESSOR_ID = os.getenv('PROCESSOR_ID', "341e773888de340f")
    PROCESSOR_VERSION = os.getenv('PROCESSOR_VERSION', "rc")
    MIME_TYPE = os.getenv('MIME_TYPE', "image/png")
    SECRET_TOKEN = os.getenv('SECRET_TOKEN', "VXZXYMCFUHpEOAE7duiIrFbKCjtTFURof6iGIbXmT1WzK05DhqdC6X2bChvdqGOt")
    
app.add_event_handler("startup", startup_event)

Overwriting service_ocr/main.py


In [7]:
# Para probar en local
# uvicorn main:app --reload

## Deply Cloud Run
- cd $DIRECTORY
- bash deploy.sh

## Test Local

- cd $DIRECTORY
- uvicorn main:app --reload

# --------TEST DE CONSUMO DEL SERVICIO--------------------

## 1. Ejemplos base

In [ ]:
# !curl https://chat-gpt-api-new-yutgchy3pa-uc.a.run.app
#!uvicorn main:app --reload

In [37]:
import requests

url = "http://127.0.0.1:8000"

#url = "https://tinka-ocr-yutgchy3pa-uc.a.run.app"

#url = "https://tinka-ocr-dw2j52225q-uc.a.run.app"

headers = {}

response = requests.get(url, headers=headers)
response.json()

{'message': 'Servicio de reconocimiento de caracteres a partir de imagen, para la Tinka',
 'model_version': 0.1}

In [7]:
import requests
import pandas as pd
import base64
import numpy as np
import cv2
import json

def service_tinka_ocr(url, image_path, pozos, id_cliente):
    
    img = cv2.imread(image_path)
    _, im_arr = cv2.imencode('.png', img)  # im_arr: image in Numpy one-dim array format.
    im_bytes = im_arr.tobytes()
    im_b64_encoded = base64.b64encode(im_bytes).decode("utf-8")
    
    #headers = {"token": "my-secret-token"}
    headers = {"token": "VXZXYMCFUHpEOAE7duiIrFbKCjtTFURof6iGIbXmT1WzK05DhqdC6X2bChvdqGOt"}
    
    data = {
            "base64_data": im_b64_encoded,
            "json_data": pozos,
            "id_cliente": id_cliente
            }
    
    response = requests.post(url, json=data, headers=headers)
    #print(response, type(response))    
    if response.status_code == 200:
        print(response.json())
        data = response.json()
        print("resultado: ", data["resultado"])
        print("tipo: ", data["tipo"])
        print("mensaje: ", data["mensaje"])
        #df = pd.DataFrame.from_dict(response.json())
        #print(df)
        
        #print(response.json())
    else:
        print("Error al enviar la imagen al servicio o token inválido")
        print(response)
    
    #return df

In [8]:
#url = "http://127.0.0.1:8000/tinka-ocr/"

#url = "https://tinka-ocr-yutgchy3pa-uc.a.run.app/tinka-ocr/"
url = "https://tinka-ocr-dw2j52225q-uc.a.run.app/tinka-ocr/"

image_path = "../images/image1.png"
pozos = {"Tinka": [float(14), float(15)],"Kabala": [float(640), None], "Ganagol": [float(700), float(700)]}
id_usuario = "234534345345"

service_tinka_ocr(url, image_path, pozos, id_usuario)

[ WARN:0@1.174] global loadsave.cpp:248 findDecoder imread_('../images/image1.png'): can't open/read file: check file path/integrity


error: OpenCV(4.8.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:1113: error: (-215:Assertion failed) !image.empty() in function 'imencode'


In [47]:
url = "http://127.0.0.1:8000/tinka-ocr/"

#url = "https://tinka-ocr-yutgchy3pa-uc.a.run.app/tinka-ocr"
#url = "https://tinka-ocr-dw2j52225q-uc.a.run.app/tinka-ocr/"

image_path = "../images/image2.png"
pozos = {"Tinka": [float(14), float(14)],"Kabala": [float(650), float(650)], "Ganagol": [float(700), float(700)]}
id_usuario = "234534345345"

service_tinka_ocr(url, image_path, pozos, id_usuario)

{'resultado': 'False', 'tipo': '2', 'mensaje': 'La foto no es válida. Enfoca todo el elemento multipozo para validarlo. [Tinka:14.0/Kabala:650.0/Ganagol:700.0]'}
resultado:  False
tipo:  2
mensaje:  La foto no es válida. Enfoca todo el elemento multipozo para validarlo. [Tinka:14.0/Kabala:650.0/Ganagol:700.0]


In [48]:
url = "http://127.0.0.1:8000/tinka-ocr/"

#url = "https://tinka-ocr-yutgchy3pa-uc.a.run.app/tinka-ocr"
#url = "https://tinka-ocr-dw2j52225q-uc.a.run.app/tinka-ocr/"

image_path = "../images/image3.png"
pozos = {"Tinka": [float(14), float(14)],"Kabala": [float(650), float(650)], "Ganagol": [float(700), float(700)]}
id_usuario = "234534345345"

service_tinka_ocr(url, image_path, pozos, id_usuario)

{'resultado': 'False', 'tipo': '4', 'mensaje': 'Los pozos son incorrectos. Los pozos correctos son [Tinka:14.0/Kabala:650.0/Ganagol:700.0]'}
resultado:  False
tipo:  4
mensaje:  Los pozos son incorrectos. Los pozos correctos son [Tinka:14.0/Kabala:650.0/Ganagol:700.0]


In [49]:
url = "http://127.0.0.1:8000/tinka-ocr/"

#url = "https://tinka-ocr-yutgchy3pa-uc.a.run.app/tinka-ocr"

image_path = "../images/image4.png"
pozos = {"Tinka": [float(14), float(14)],"Kabala": [float(650), float(650)], "Ganagol": [float(700), float(700)]}
id_usuario = "234534345345"

service_tinka_ocr(url, image_path, pozos, id_usuario)

{'resultado': 'False', 'tipo': '1', 'mensaje': 'La foto no es válida.  Toma una foto del elemento multipozo. [Tinka:14.0/Kabala:650.0/Ganagol:700.0]'}
resultado:  False
tipo:  1
mensaje:  La foto no es válida.  Toma una foto del elemento multipozo. [Tinka:14.0/Kabala:650.0/Ganagol:700.0]


In [50]:
url = "http://127.0.0.1:8000/tinka-ocr/"

#url = "https://tinka-ocr-yutgchy3pa-uc.a.run.app/tinka-ocr"

image_path = "../images/image5.png"
pozos = {"Tinka": [float(14), float(14)],"Kabala": [float(650), float(650)], "Ganagol": [float(700), float(700)]}
id_usuario = "234534345345"

service_tinka_ocr(url, image_path, pozos, id_usuario)

{'resultado': 'True', 'tipo': '1', 'mensaje': 'OK los pozos son correctos. Tinka: 14.0; Kabala:650.0; Ganagol:700.0'}
resultado:  True
tipo:  1
mensaje:  OK los pozos son correctos. Tinka: 14.0; Kabala:650.0; Ganagol:700.0


In [51]:
url = "http://127.0.0.1:8000/tinka-ocr/"

#url = "https://tinka-ocr-yutgchy3pa-uc.a.run.app/tinka-ocr"

image_path = "../images/image6.png"
pozos = {"Tinka": [float(14), float(14)],"Kabala": [float(650), float(650)], "Ganagol": [float(700), float(700)]}
id_usuario = "234534345345"

service_tinka_ocr(url, image_path, pozos, id_usuario)

{'resultado': 'False', 'tipo': '3', 'mensaje': 'La foto no es válida. Toma una foto del elemento multipozo de tu Punto de Venta. [Tinka:14.0/Kabala:650.0/Ganagol:700.0]'}
resultado:  False
tipo:  3
mensaje:  La foto no es válida. Toma una foto del elemento multipozo de tu Punto de Venta. [Tinka:14.0/Kabala:650.0/Ganagol:700.0]


## 2. Pruebas de los Errores

In [19]:
import requests
import pandas as pd
import base64
import numpy as np
import cv2
import json

def service_tinka_ocr(url, image_path, pozos, id_cliente):
    
    img = cv2.imread(image_path)
    _, im_arr = cv2.imencode('.png', img)  # im_arr: image in Numpy one-dim array format.
    im_bytes = im_arr.tobytes()
    im_b64_encoded = base64.b64encode(im_bytes).decode("utf-8")
    
    #headers = {"token": "my-secret-token"}
    headers = {"token": "VXZXYMCFUHpEOAE7duiIrFbKCjtTFURof6iGIbXmT1WzK05DhqdC6X2bChvdqGOt"}
    
    data = {
            "base64_data": im_b64_encoded,
            "json_data": pozos,
            "id_cliente": id_cliente
            }
    
    response = requests.post(url, json=data, headers=headers)
    #print(response, type(response))    
    if response.status_code == 200:
        print(response.json())
        data = response.json()
        print("resultado: ", data["resultado"])
        print("tipo: ", data["tipo"])
        print("mensaje: ", data["mensaje"])
        #df = pd.DataFrame.from_dict(response.json())
        #print(df)
        
        #print(response.json())
    else:
        print("Error al enviar la imagen al servicio o token inválido")
        print(response)
    
    #return df

In [21]:
url = "http://127.0.0.1:8000/tinka-ocr/"

#url = "https://tinka-ocr-m5leg4psgq-uc.a.run.app/tinka-ocr/"

#image_path = "../images_test/error-09-10/image7.jpg"

image_path = "../images_test/error27/image3_3.jpg"
#image_path = "../images_test/error19/4.jpg"

#pozos = {"Tinka": [11, 11],"Kabala": [1.5, 1.5], "Ganagol": [1.5, 1.5]}
#pozos = {"Tinka": [float(11), float(11)],"Kabala": [float(1.5), float(1.5)], "Ganagol": [float(1.5), float(1.5)]}

#pozos = {"Tinka": [14, 14],"Kabala": [2, 2], "Ganagol": [1.5, 1.5]}
#pozos = {"Tinka": [float(14), float(14)],"Kabala": [float(2), float(2)], "Ganagol": [float(1.5), float(1.5)]}

pozos = {"Tinka": [13, 13],"Kabala": [2, 2], "Ganagol": [1.5, 1.5]}
#pozos = {"Tinka": [13, 13],"Kabala": [2.0, 2.0], "Ganagol": [1.5, 1.5]}
#pozos = {"Tinka": [float(13), float(13)],"Kabala": [float(2), float(2)], "Ganagol": [float(1.5), float(1.5)]}

#pozos = {"Tinka": [13, 13],"Kabala": [2, 2], "Ganagol": [1.5, 1.5]}
#pozos = {"Tinka": [15, 15],"Kabala": [2.0, 2.0], "Ganagol": [1.5, 1.5]}
#pozos = {"Tinka": [16, 16],"Kabala": [2.5, 2.5], "Ganagol": [1.5, 1.5]}
   
#pozos = {"Tinka": [5.5, 5.5],"Kabala": [400, 400], "Ganagol": [1.5, 1.5]}

id_usuario = "234534345345"

service_tinka_ocr(url, image_path, pozos, id_usuario)

{'resultado': 'True', 'tipo': '1', 'mensaje': 'OK los pozos son correctos. Tinka: 13; Kabala:2; Ganagol:1.5'}
resultado:  True
tipo:  1
mensaje:  OK los pozos son correctos. Tinka: 13; Kabala:2; Ganagol:1.5


In [16]:
import re


#cadena = "El precio es $12.34 y el descuento es de $5.67. Además, hay 100 unidades disponibles."
cadena = "2.0="

# Utiliza una expresión regular para encontrar números enteros y números con punto decimal
numeros = re.findall(r'\d+\.\d+|\d+', cadena)

# Imprime los números encontrados
for numero in numeros:
    print(numero)


2.0


In [34]:
import math

print("{:0.2f}".format(float("  45.573   ")))

45.57


In [32]:
import re

cadena = "H0l4 M* Mundo 123!"

# Utiliza una expresión regular para reemplazar todos los caracteres que no son dígitos por una cadena vacía
cadena_sin_no_numeros = re.sub(r'[^0-9]', '', cadena)
print(cadena_sin_no_numeros)
print(len(cadena), len(cadena_sin_no_numeros))
# Calcula la longitud de la cadena resultante
conteo_caracteres_no_numeros = len(cadena) - len(cadena_sin_no_numeros)

print("Cantidad de caracteres no numéricos:", conteo_caracteres_no_numeros)

04123
18 5
Cantidad de caracteres no numéricos: 13


In [51]:
palabra = "er3rr45="

def verificar_numero_valido(text):
    count = 0
    for c in palabra:
        #print(c.isnumeric())
        if (count >3):
            return False
        
        if not c.isnumeric() and c!=' ':
            count+=1
    return True
    
verificar_numero_valido(palabra)

False

In [25]:
#palabra in palabra_contenedora
"Actualiza".lower() in "aaa asdsadACTUALIZA los montOsa$34 3445".lower()

True